In [71]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE

%matplotlib inline

In [72]:
def submit(preds, test_feature, label_map):
    submission = test_feature[['id']]
    submission['pred'] = [label_map[l] for l in np.nditer(preds)]
    submission.to_csv('./predicts/0217_1_partition.csv', header=False, index=False)
    return submission
    

In [73]:
test_feature = pd.read_csv('./test_feature_line_index_0217.csv',header=None)
test_feature.head()
train_feature = pd.read_csv('./train_feature_line_index_0217.csv',header=None)
test_feature.rename(columns={0:'id'}, inplace=True)
train_feature.head()


,0,1,2,3,4,5,6,7,8,9,...,112,113,114,115,116,117,118,119,120,121
0,696220,2807192.6,1206.6,4833.9,-362.9,3137136.9,4.0,2326.5,-0.3,380.9,...,1.5,419.6,3142.2,3516.1,2697.3,109977.9,1.1,0.1,0.9,star
1,911466,899.3,112.2,166.7,-2.9,291801.1,1.5,8.0,-0.0,1.0,...,0.8,1.1,106.6,125.3,82.4,3730.8,1.2,0.0,0.8,star
2,915856,434.2,61.4,126.5,24.1,159725.3,2.1,7.1,0.4,0.2,...,0.8,0.1,81.3,88.6,76.2,2845.1,1.1,0.0,0.9,star
3,720026,17101718.7,12073.5,20181.5,3666.1,31391013.8,1.7,1416.5,0.3,4507.3,...,0.9,4272.4,16433.0,17430.5,14962.0,575156.0,1.1,0.3,0.9,star
4,1198160,11729.1,379.8,491.7,-225.7,987376.9,1.3,30.9,-0.6,10.4,...,0.9,10.6,359.2,410.2,273.5,12572.3,1.1,0.0,0.8,star


In [65]:
columns = {0:'id'}
columns[train_feature.shape[1]-1] = 'label'
train_feature.rename(columns=columns, inplace=True)
train_feature.head()


,id,1,2,3,4,5,6,7,8,9,...,77,78,79,80,81,82,83,84,85,label
0,696220,1261.5,4833.9,-362.9,431.0,2751624.3,3007.4,174.4,3160.9,1762.5,...,0.1,-107.8,0.2,-102.9,0.1,-100.2,0.1,-93.8,0.3,star
1,911466,111.2,166.7,-2.9,1137.0,892.7,20.7,0.1,29.4,1.4,...,0.0,99.1,0.1,95.3,0.1,92.9,0.0,89.4,0.1,star
2,915856,62.2,126.5,24.1,232.0,394.6,79.9,0.3,86.6,0.2,...,0.0,34.6,0.0,33.8,0.0,33.4,0.0,31.9,0.0,star
3,720026,12172.8,20181.5,3666.1,431.0,15412356.1,13195.2,15871.1,14468.2,25726.1,...,150.3,6747.5,143.9,6521.3,77.8,6337.3,63.1,6164.4,30.6,star
4,1198160,373.0,491.7,-225.7,1512.0,13211.7,-1.4,5.9,40.6,8.7,...,0.1,396.4,0.9,391.0,0.3,392.6,0.0,392.0,0.4,star


In [66]:
train_feature.groupby('label').agg(['count'])

,id,1,2,3,4,5,6,7,8,9,...,76,77,78,79,80,81,82,83,84,85
,count,count,count,count,count,count,count,count,count,count,...,count,count,count,count,count,count,count,count,count,count
label,,,,,,,,,,,,,,,,,,,,,
galaxy,5231,5231,5231,5231,5231,5231,5231,5231,5231,5231,...,5231,5231,5231,5231,5231,5231,5231,5231,5231,5231
qso,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363,...,1363,1363,1363,1363,1363,1363,1363,1363,1363,1363
star,442969,442969,442969,442969,442969,442969,442969,442969,442969,442969,...,442969,442969,442969,442969,442969,442969,442969,442969,442969,442969
unknown,34288,34288,34288,34288,34288,34288,34288,34288,34288,34288,...,34288,34288,34288,34288,34288,34288,34288,34288,34288,34288


In [67]:
train_star = train_feature[train_feature.label=='star'].sample(34000, replace=False)
unknow_star = train_feature[train_feature.label=='unknown'].sample(34000, replace=False)
galaxy_star = train_feature[train_feature.label=='galaxy'].sample(34000, replace=True)
qso_star = train_feature[train_feature.label=='qso'].sample(34000, replace=True)
sampled_train = pd.concat([train_star,unknow_star,galaxy_star, qso_star])

In [68]:
features = train_feature.columns[1:-1]
x= sampled_train[features]
y, label_map = pd.factorize(sampled_train['label'])
t_x =test_feature.iloc[:,1:]

In [69]:
clf = RandomForestClassifier(n_jobs=4)
# clf = svm.SVC(kernel='rbf')

In [70]:
rs = clf.fit(x, y)
preds = clf.predict(test_feature.iloc[:,1:])


In [50]:
sb = submit(preds, test_feature, label_map)
sb.pred.value_counts()

/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


star       78550
unknown    10753
galaxy      7484
qso         3213
Name: pred, dtype: int64